In [ ]:
%matplotlib widget
import ROOT

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy

In [ ]:
# Constants
_cLIGHT = 299792458   # [m/s]
_nsTOs = 1e-9         # [s/ns]

# Detector Size
_L = 2                # [m] (Flight Length)

## Load the Data

In [ ]:
FILENAME = "./MixedParticles_N100000.root"

rdf = ROOT.RDataFrame('Events', f"./{FILENAME}")
data = rdf.AsNumpy(["Time0", "Time1", "TrueMomentum", "TruePID", "TrueTime0", "TrueTime1"])

dT = (data["Time1"] - data["Time0"])*_nsTOs
trueMomentum = data["TrueMomentum"]
momentum = trueMomentum # + np.random.normal(scale=0.05, size=len(dT)) # With potentially added momentum uncertainty

truePID = data["TruePID"]
truedT = (data["TrueTime1"] - data["TrueTime0"])*_nsTOs

### Initial Filtering (data cuts)

In [ ]:
# Remove samples with negative flight time
select_dT0 = dT > 0

dT = dT[select_dT0]
momentum = momentum[select_dT0]

trueMomentum = trueMomentum[select_dT0]
truePID = truePID[select_dT0] 
truedT = truedT[select_dT0]

## Calculate the Particle Speed

In [ ]:
beta = _L/dT/_cLIGHT

# Particle Identification

## Overview Plot

**Can you attribute the traces to the five simulated particles: e-, mu-, pi+, K+ and p?**

**What is the property which mainly distinguishes them?**

In [ ]:
fig, (axlin, axlog) = plt.subplots(1, 2)
fig.set_figwidth(10)

# Linear Scale Plot
axlin.hist2d(momentum, beta, 300)
axlin.set_xlabel('Momentum [GeV/c]')
axlin.set_ylabel('beta [-]')
axlin.set_title('Linear Color Scale')

# Log Scale Plot
axlog.hist2d(momentum, beta, 300, norm=mpl.colors.LogNorm())
axlog.set_xlabel('Momentum [GeV/c]')
axlog.set_ylabel('beta [-]')
axlog.set_title('Log Color Scale')

plt.show()

## Mass Reconstruction

### Filter the undistinguishable region (beta ~ 1)

In [ ]:
select_beta = beta < 0.92

select_mass = select_beta

beta_mass = beta[select_mass]
momentum_mass = momentum[select_mass]

### Estimate the particle masses

In [ ]:
gamma_mass = 1/np.sqrt(1 - beta_mass**2)

m0 = momentum_mass / gamma_mass / beta_mass

fig = plt.figure()
fig.set_figwidth(10)
plt.hist(m0, 500, range=(0, 1.5))
plt.xlabel('Estimated Mass [GeV/c²]')
plt.ylabel('Count [-]')
plt.show()

### Fit the Mass Peaks

In [ ]:
def gaussPeak(m, count, m_avg, m_sig):
    peak = scipy.stats.norm.pdf(m_avg, m_avg, m_sig)
    
    return scipy.stats.norm.pdf(m, m_avg, m_sig)/peak*count

def fitMass(mass_centers, counts, lower, upper):
    select_range = (mass_centers > lower) & (mass_centers < upper)
    
    mass_centers = mass_centers[select_range]
    counts = counts[select_range]
    
    p0 = (np.max(counts), mass_centers[np.argmax(counts)], 0.1)

    popt, pcov = scipy.optimize.curve_fit(gaussPeak, mass_centers, counts, p0=p0)

    return popt

def plotPeak(ax, lower, upper, popt, label=None):
    masses = np.linspace(lower, upper, 100)
    ax.plot(masses, gaussPeak(masses, *popt), label=label)

fig = plt.figure()
counts, edges, _ = plt.hist(m0, 500, range=(0, 1.5))
mass_centers = (edges[:-1] + edges[1:])/2

popt = fitMass(mass_centers, counts, 0.92, 0.98)

plotPeak(fig.gca(), 0.75, 1.2, popt, 'Proton')


fig.set_figwidth(10)
plt.xlabel('Estimated Mass [GeV/c²]')
plt.ylabel('Count [-]')
plt.yscale('log')
plt.ylim(10, 1000)
plt.show()

**Why does the mass not follow a normal distribution?**

## Mass Distributions with an Ideal Detector

### Calculate beta and mass based on ideal / true values

In [ ]:
trueBeta = _L/truedT/_cLIGHT

select_beta = trueBeta < 0.99

truePID = truePID[select_beta]
trueBeta = trueBeta[select_beta]
trueMomentum = trueMomentum[select_beta]

In [ ]:
trueGamma = 1/np.sqrt(1 - trueBeta**2)
trueM0 = trueMomentum / trueGamma / trueBeta

In [ ]:
plt.figure()

plt.hist(trueM0, bins=1000, range=(0.,2))

plt.xlabel('Estimated Mass [GeV/c²]')
plt.ylabel('Count [-]')
plt.yscale('log')
plt.show()

### Plot separated by Particle Type

In [ ]:
# Plot all particles as separate histograms
uniquePID = np.unique(truePID)
plt.figure()
for p in uniquePID:
    sel = (truePID == p)
    plt.hist(trueM0[sel], bins=1000, range=(0.,2), histtype='step', label=p)

plt.legend()
plt.yscale('log')
plt.show()

**What do these numbers mean in the legend?**

**Why does the proton (2212) have tail towards higher masses?**

**Why does Kaon (K+) have a tail towards lower masses?** 